In [12]:
# Starting again, trying to solve it all...

import torch
import torch.nn as nn
import scipy.optimize as opt
import numpy as np
import sys
sys.path.append("../")
from ddn.pytorch.node import *
import warnings
warnings.filterwarnings('ignore')

class NormalizedCuts(EqConstDeclarativeNode):
    """
    A declarative node to embed Normalized Cuts into a Neural Network
    
    Normalized Cuts and Image Segmentation https://people.eecs.berkeley.edu/~malik/papers/SM-ncut.pdf
    Shi, J., & Malik, J. (2000)
    """
    def __init__(self):
        super().__init__()
        
    def objective(self, x, y):
        """
        f(W,y) = y^T * (D-W) * y / y^T * D * y
        """
        # W is an NxN symmetrical matrix with W(i,j) = w_ij
        D = x.sum(1).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
        L = D - x
        
        y_t = torch.t(y)
        
        return torch.div(torch.matmul(torch.matmul(y_t, L),y),torch.matmul(torch.matmul(y_t,D),y))
    
    def equality_constraints(self, W, y):
        """
        subject to y^T * D * 1 = 0
        """
        # Ensure correct size and shape of y... scipy minimise flattens y         
        N = W.size(dim=0)
        
        #x is an NxN symmetrical matrix with W(i,j) = w_ij
        D = W.sum(1).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
        ONE = torch.ones(N,1)   # Nx1 vector of all ones
        
        return torch.mm(torch.mm(torch.t(y),D), ONE)

    def solve(self, W):
        """
        Minimise the objective, by solving the second smallest eigenvector of laplacian
        (D-W)*y = lambda * D * y
        becomes D^-0.5 * (D-W) * D^-0.5 * y = lambda * y
        """
        # normalised laplacian
        D = W.sum(1).diag()
        L = D - W
        # Solve using torch.linalg.eigh?
        eigval, eigvec = torch.linalg.eig(L)

        val = [np.round(i.real,4) for i in eigval]

        vec = []
        for i in range(len(eigvec)):
            v = [np.round(n.real,4) for n in eigvec[:,i]]
            vec.append(v)

        s = list(val).copy()
        s.sort()

        for i in range(len(eigval)):
            if val[i] == s[1]:
                second_smallest_eigval_vec = vec[i]
                break
        
        return torch.tensor(second_smallest_eigval_vec), _
    
node = NormalizedCuts()
x = torch.tensor([[0,1,0], [2,0,3], [0,4,0]]).double()
y, misc = node.solve(x)
# node.gradient(x)
print(y)
print("done")
# torch.cuda.set_device(gpu)
# model = model.cuda(gpu)

tensor([-0.4937,  0.4136,  0.7650], dtype=torch.float64)
done
